In [24]:
from tomok.ifc import IFCReader

In [25]:
reader = IFCReader(ifc_filepath="/Users/jaewooklee/Documents/Github/tomok/tomok/ED_extension_usingToolkit_v2.ifc")

In [177]:
reader.get_products()
e = reader.get_product_by_guid(guid="3s45s2y_vBWuTHL$ihTpyq")
e

#206896=IfcBearing('3s45s2y_vBWuTHL$ihTpyq',#18,'Column-1020',$,$,#206895,#206892,'155815',.ELASTOMERIC.)

In [182]:
e['KRPset_KDS 14 20 54_4.3.5 (1)']["The distance from the center of the anchor to the edge of the projected influence area required to develop the maximum bond strength"]

'-'

In [158]:
from tomok.core.rule_ifc import RuleIFC
from tomok.ifc.reader import IFCReader
from tomok.ifc.entity import Product

from ruleunits import (
    kds142054_040305_01,
    kds142054_040305_03,
    kds142054_040305_04,
    kds142054_040305_05
)


class RuleIFC_KDS142054_01(RuleIFC):
    # RuleIFC
    def __init__(self,
                 ifc_filename: str):
        self.reader = IFCReader(ifc_filename) # IFCReader 생성
        self.runit = {
            '01': kds142054_040305_01.KDS142054_040305_01(),
            '03': kds142054_040305_03.KDS142054_040305_03(),
            '04': kds142054_040305_04.KDS142054_040305_04(),
            '05': kds142054_040305_05.KDS142054_040305_05()
        }

    def retrieve_entities(self,
                          guid: str):
        self.reader.get_products() # IFC 파일에서 부재 목록을 읽어옵니다.
        try:
            target_entity = [self.reader.get_product_by_guid(guid)] # guid를 기반으로 부재를 검색합니다.
        except:
            return []
        return target_entity

    def _get_p_value(self,
                     entity: Product,
                     krpset_name: str,
                     en_name: str):
        try:
            return entity[krpset_name][en_name]
        except:                
            return -1
        
    def process(self,
                entity: Product):
        try:
            # KDS142054_040305_03
            # fOpsecNa = self._get_p_value(entity=entity,
            #                             krpset_name="KRPset_KDS 14 20 54_4.3.5 (3)",
            #                             en_name="Correction factor for attached anchor groups in which eccentricity of tensile force acts")
            fIeprimN = self._get_p_value(entity=entity,
                                        krpset_name="KRPset_KDS 14 20 54_4.3.5 (3)",
                                        en_name="The distance between the resultant force of the tensile force acting on the anchor group subjected to tensile load and the centroid of the anchor group")
            fIcNa = self._get_p_value(entity=entity,
                                      krpset_name="KRPset_KDS 14 20 54_4.3.5 (3)",
                                      en_name="The distance from the center of the anchor to the edge of the projected influence area required to develop the maximum bond strength")
            fOpsecNa = self.runit['03'].correction_factor_for_attached_anchor_groups_with_eccentricity_of_tensile_force(0, fIeprimN, fIcNa)
            # KDS142054_040305_04
            # fOpsedNa = self._get_p_value(entity=entity,
            #                             krpset_name="KRPset_KDS 14 20 54_4.3.5 (4)",
            #                             en_name="Correction factor for edge influence of single anchor anchor or group of anchor anchors")
            fIcamin = self._get_p_value(entity=entity,
                                    krpset_name="KRPset_KDS 14 20 54_4.3.5 (4)",
                                    en_name="Minimum edge distance from anchor shaft center to concrete end")
            fIcNa = self._get_p_value(entity=entity,
                                    krpset_name="KRPset_KDS 14 20 54_4.3.5 (4)",
                                    en_name="The distance from the center of the anchor to the edge of the projected influence area required to develop the maximum bond strength")
            fOpsedNa = self.runit['04'].correction_factor_for_edge_influence_of_single_or_group_of_anchor(0, fIcamin, fIcNa)
            # KDS142054_040305_05
            # fOpscpNa = self._get_p_value(entity=entity,
            #                             krpset_name="KRPset_KDS 14 20 54_4.3.5 (5)",
            #                             en_name="Modification factors for bonded anchors used in uncracked concrete")
            fIcamin = self._get_p_value(entity=entity,
                                    krpset_name="KRPset_KDS 14 20 54_4.3.5 (5)",
                                    en_name="Minimum edge distance from anchor shaft center to concrete end")
            fIcac = self._get_p_value(entity=entity,
                                    krpset_name="KRPset_KDS 14 20 54_4.3.5 (5)",
                                    en_name="risk podium distance")
            fOpscpNa = self.runit['05'].modification_factors_for_bonded_anchors_used_in_uncracked_concrete(0, fIcamin, fIcac)

            # KDS142054_040305_01
            fINa = self._get_p_value(entity=entity,
                                     krpset_name="KRPset_KDS 14 20 54_4.3.5 (1)",
                                     en_name="Nominal bond strength of a single bonded anchor")
            fINag = self._get_p_value(entity=entity,
                                      krpset_name="KRPset_KDS 14 20 54_4.3.5 (1)",
                                      en_name="Nominal bond strength of adhesive anchor group")
            fIANa = self._get_p_value(entity=entity,
                                      krpset_name="KRPset_KDS 14 20 54_4.3.5 (1)",
                                      en_name="Projected influence area of anchor group")
            fIANao = self._get_p_value(entity=entity,
                                       krpset_name="KRPset_KDS 14 20 54_4.3.5 (1)",
                                       en_name="Projected influence area of a single anchor")
            fIpsedNa = fOpsedNa # KDS142054_040305_04 실행 결과
            fIpscpNa = fOpscpNa # KDS142054_040305_05 실행 결과
            fINba = self._get_p_value(entity=entity,
                                    krpset_name="KRPset_KDS 14 20 54_4.3.5 (1)",
                                    en_name="Basic bond strength of a single bonded anchor in tension")
            fIpsecNa = fOpsecNa # KDS142054_040305_03 실행 결과
            fIn = self._get_p_value(entity=entity,
                                krpset_name="KRPset_KDS 14 20 54_4.3.5 (1)",
                                en_name="Number of Attachable Anchors")
            fIcNa = self._get_p_value(entity=entity,
                                    krpset_name="KRPset_KDS 14 20 54_4.3.5 (1)",
                                    en_name="The distance from the center of the anchor to the edge of the projected influence area required to develop the maximum bond strength")
            fIda = self._get_p_value(entity=entity,
                                    krpset_name="KRPset_KDS 14 20 54_4.3.5 (1)",
                                    en_name="The outer diameter of an anchor, or the shaft diameter of a head stud, head bolt, or claw bolt")
            fItauncr = self._get_p_value(entity=entity,
                                        krpset_name="KRPset_KDS 14 20 54_4.3.5 (1)",
                                        en_name="Characteristics Adhesion strength of adhesive anchors used in uncracked concrete")
            fIuserdefined = 1
            result = self.runit['01'].nominal_bond_strength_of_a_single_bonded_anchor(fINa,fINag,fIANa,fIANao,fIpsedNa,fIpscpNa,fINba,fIpsecNa,fIn,fIcNa,fIda,fItauncr,fIuserdefined)

            print(result)
            print("process end.")
        except:
            print("Fail")

In [159]:
ruleifc = RuleIFC_KDS142054_01("/Users/jaewooklee/Documents/Github/tomok/tomok/ED_extension_usingToolkit_v2.ifc")

In [184]:
entity = ruleifc.retrieve_entities("3s45s2y_vBWuTHL$ihTpyq")[0]

In [185]:
print(entity["KRPset_KDS 14 20 54_4.3.5 (1)"]["The distance from the center of the anchor to the edge of the projected influence area required to develop the maximum bond strength"])
print(ruleifc._get_p_value(entity, "KRPset_KDS 14 20 54_4.3.5 (1)", 'The distance from the center of the anchor to the edge of the projected influence area required to develop the maximum bond strength'))

-
-


In [193]:
entity = ruleifc.reader.get_products('IFCBearing')[109]
ruleifc.process(entity)

Pass
process end.
